# DeepSDF
![](imgs/intro.png)

Slide credits: DeepSDF (Park et. al.), Minhyuk Sung (CS492(A): Machine Learning for 3D Data, KAIST, Spring 2022)

![](imgs/1.png)

![](imgs/2.png)

![](imgs/3.png)

![](imgs/4.png)

![](imgs/0.png)

![](imgs/5.png)

![](imgs/6.png)

![](imgs/7.png)

In [ ]:
! pip install fvcore iopath

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=e790562dba9a42c3321c40097e2b5acbef28e6b105cbda6d0069f001b0b543cb
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=450286524ba41a73a6bb6716289554190aeca41f626c322d2025ca7d413b5d33
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [ ]:
! git clone https://github.com/niladridutt/DeepSDF.git
! mv DeepSDF/* DeepSDF/.* . 2>/dev/null && rmdir DeepSDF

--2025-01-29 00:53:20--  https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt251/pytorch3d-0.7.8-cp310-cp310-linux_x86_64.whl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.15, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20528145 (20M) [binary/octet-stream]
Saving to: ‘pytorch3d-0.7.8-cp310-cp310-linux_x86_64.whl’

pytorch3d-0.7.8-cp3 100%[===================>]  19.58M   127MB/s    in 0.2s    

2025-01-29 00:53:21 (127 MB/s) - ‘pytorch3d-0.7.8-cp310-cp310-linux_x86_64.whl’ saved [20528145/20528145]



In [ ]:
! pip install pytorch3d-0.7.8-cp312-cp312-linux_x86_64.whl
# ! pip install "git+https://github.com/facebookresearch/pytorch3d.git@stable" from source

! pip install trimesh
! pip install pybullet
! pip install point-cloud-utils


Processing ./pytorch3d-0.7.8-cp310-cp310-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 707.0/707.0 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 22.2 MB/s eta 0:00:00


In [4]:
import torch
import pytorch3d

In [8]:
#! python extract_sdf.py

In [9]:
import torch
import os
import model.model_sdf as sdf_model
from utils import utils_deepsdf
import trimesh
from results import runs_sdf
import numpy as np
import yaml

# Set device for computations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Function: Read Parameters
# This block defines a function to read training settings from a configuration file.
def read_params(cfg):
    training_settings_path = os.path.join("./results/runs_sdf", cfg['folder_sdf'], 'settings.yaml')
    with open(training_settings_path, 'rb') as f:
        training_settings = yaml.load(f, Loader=yaml.FullLoader)

    return training_settings


In [11]:
def reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis):
    """
    Reconstruct the object from the latent code and save the mesh.
    Meshes are stored as .obj files under the same folder cerated during training, for example:
    - runs_sdf/<datetime>/meshes_training/mesh_0.obj
    """
    sdf = utils_deepsdf.predict_sdf(latent_code, coords_batches, model)
    try:
        vertices, faces = utils_deepsdf.extract_mesh(grad_size_axis, sdf)
    except:
        print('Mesh extraction failed')
        return

    # save mesh as obj
    mesh_dir = os.path.join("./results/runs_sdf", cfg['folder_sdf'], 'meshes_training')
    if not os.path.exists(mesh_dir):
        os.mkdir(mesh_dir)
    obj_path = os.path.join(mesh_dir, f"mesh_{obj_idx}.obj")
    trimesh.exchange.export.export_mesh(trimesh.Trimesh(vertices, faces), obj_path, file_type='obj')


In [12]:
cfg_path = './config_files/reconstruct_from_latent.yaml'
with open(cfg_path, 'rb') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

In [17]:
cfg['obj_ids'] = ['02942699/5d42d432ec71bfa1d5004b533b242ce6']
# cfg['obj_ids'] = ['02942699/5d42d432ec71bfa1d5004b533b242ce6', '02876657/e9371d3abbb3bb7265bca0cae1ecfff5','03797390/ea127b5b9ba0696967699ff4ba91a25']

In [19]:
training_settings = read_params(cfg)

# Load the model
weights = os.path.join("./results/runs_sdf", cfg['folder_sdf'], 'weights.pt')

model = sdf_model.SDFModel(
    num_layers=training_settings['num_layers'],
    skip_connections=training_settings['latent_size'],
    latent_size=training_settings['latent_size'],
    inner_dim=training_settings['inner_dim']).to(device)
model.load_state_dict(torch.load(weights, map_location=device))

# Extract mesh obtained with the latent code optimised at inference
coords, grad_size_axis = utils_deepsdf.get_volume_coords(cfg['resolution'])
coords = coords.to(device)

# Split coords into batches because of memory limitations
coords_batches = torch.split(coords, 100000)

# Load paths
str2int_path = os.path.join("./results/runs_sdf", 'idx_str2int_dict.npy')
results_dict_path = os.path.join("./results/runs_sdf", cfg['folder_sdf'], 'results.npy')

# Load dictionaries
str2int_dict = np.load(str2int_path, allow_pickle=True).item()
results_dict = np.load(results_dict_path, allow_pickle=True).item()

for obj_id_path in cfg['obj_ids']:
    # Get object index in the results dictionary
    obj_idx = str2int_dict[obj_id_path]  # index in collected latent vector
    # Get the latent code optimised during training
    latent_code = results_dict['best_latent_codes'][obj_idx]
    latent_code = torch.tensor(latent_code).to(device)

    reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis)



<ipython-input-19-b962ccc9ef1b>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(weights, map_location=device))


In [20]:
import trimesh

mesh = trimesh.load('./results/runs_sdf/17_07_172540/meshes_training/mesh_2.obj')

mesh.show()
